In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

In [2]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,2], inplace = True)
labelsTrain.columns = [ "eye_color" , "img_name"]
labelsTrain = labelsTrain.astype({'eye_color': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,2], inplace = True)
labelsTest.columns = [ "eye_color" , "img_name"]
labelsTest = labelsTest.astype({'eye_color': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [ ]:
#230:270,180:320

In [28]:
#Load training set################################
imagesTrain = np.zeros((10000, 50 , 50, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[230:270,180:320] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = image
    
imagesTrain = imagesTrain.reshape(10000,7500)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 50 , 50, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[230:270,180:320]
    image = cv.resize(image, dsize=(50 , 50), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = image
    
    
imagesTest = imagesTest.reshape(2500,7500)   
imagesTest = pd.DataFrame(imagesTest) 

In [58]:
x_train=np.zeros((10000,7502))
for i in range(0,10000):
    x_train[i]=np.convolve(imagesTrain.iloc[i,:],[0.333,-0.333,0.333])
    
x_test=np.zeros((2500,7502))
for i in range(0,2500):
    x_test[i]=np.convolve(imagesTest.iloc[i,:],[0.333,-0.333,0.333])

In [86]:
#accuracy = []
#r=range(1000,1500,100) #peaks at 950/1000
#for i in r: 
#might be failing due to bgr values
model = KNeighborsClassifier(n_neighbors=280, weights = 'distance' , n_jobs=-1)
model.fit(x_train,labelsTrain['eye_color'])
model.score(x_test,labelsTest['eye_color'])
#accuracy.append(model.score(imagesTest,labelsTest['eye_color']))
#plt.plot(r,accuracy)

# try and seperate colours into more clear identifiers

0.8308

In [9]:
model = tree.DecisionTreeClassifier(max_depth=4,criterion = 'gini')
model.fit(imagesTrain,labelsTrain['eye_color'])
model.score(imagesTest,labelsTest['eye_color'])

0.832